In [3]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 14:21:56


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('BG_final_project').getOrCreate()

Books_rating

In [5]:
rating = spark.read.option('header', True)\
                   .csv('Books_rating.csv')

In [6]:
rating.count()

3000000

In [7]:
rating.show()

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|1882931173|Its Only Art If I...| null| AVCGYZL8FQQTD|"Jim of Oz ""jim-...|               7/7|         4.0|  940636800|Nice collection o...|This is only for ...|
|0826414346|Dr. Seuss: Americ...| null|A30TK6U7DNS82R|       Kevin Killian|             10/10|         5.0| 1095724800|   Really Enjoyed It|I don't care much...|
|0826414346|Dr. Seuss: Americ...| null|A3UH4UZ4RSVO82|        John Granger|             10/11|         5.0| 1078790400|Essential for eve...|"If people become...|
|0826414346|Dr. Seuss: Ameri

In [8]:
rating = rating.drop("Price", "profileName", "review/helpfulness",\
                     "review/summary", "review/text", "review/time")

In [9]:
# drop rows with missing value of Title or User_id
rating = rating.na.drop(how='any', subset=["Title", "User_id"])
# rating.count()

In [10]:
# drop duplicate rows
rating = rating.distinct()
# rating.count()

In [11]:
# select the data with valid raview/score
sc = ["1.0", "2.0", "3.0", "4.0", "5.0"]
rating = rating.filter(col("review/score").isin(sc))
# rating.count()

In [12]:
rating.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- review/score: string (nullable = true)



In [15]:
users=rating.select("User_id").groupBy("User_id").count()

In [18]:
#users.count()

1004214

In [23]:
users=users.filter("count>=20")
#users.count()

9355

In [24]:
users = users.alias("user")
rating = rating.alias("rate")

In [34]:
rates = rating.join(users, col("rate.User_id") == col("user.User_id"), "inner")
rates = rates.select(col("rate.User_id"), col("rate.Title"),col("rate.review/score"))

In [27]:
#rates.count()

517674

In [12]:
'''#There are review/score with null value in User_id check if need to drop those rows later
rating.groupBy("Id","Title","Price","User_id","review/helpfulness","review/score","review/time")\
    .count().filter("count > 1").sort("Id").show()'''


'#There are review/score with null value in User_id check if need to drop those rows later\nrating.groupBy("Id","Title","Price","User_id","review/helpfulness","review/score","review/time")    .count().filter("count > 1").sort("Id").show()'

In [13]:
# rating.sort(rating["review/score"].desc()).show()

In [35]:
# change type of review/score from str to int
from pyspark.sql.types import IntegerType
rates= rates.withColumn("review/score", rates["review/score"].cast(IntegerType()))

In [36]:
rates.show()

+--------------+--------------------+------------+
|       User_id|               Title|review/score|
+--------------+--------------------+------------+
|A101DG7P9E26PW|Cat's cradle (A D...|           4|
|A101DG7P9E26PW|        Cat's Cradle|           4|
|A101DG7P9E26PW|Anna Karenina: Te...|           4|
|A101DG7P9E26PW|        Cat's Cradle|           4|
|A101DG7P9E26PW|      Fahrenheit 451|           5|
|A101DG7P9E26PW|             Herland|           4|
|A101DG7P9E26PW|             Herland|           4|
|A101DG7P9E26PW|Sister to the Sio...|           2|
|A101DG7P9E26PW|Me Talk Pretty On...|           5|
|A101DG7P9E26PW|Black Elk Speaks:...|           5|
|A101DG7P9E26PW|             Herland|           4|
|A101DG7P9E26PW|             Passing|           4|
|A101DG7P9E26PW|        Cat's Cradle|           4|
|A101DG7P9E26PW|Fahrenheit 451 (C...|           5|
|A101DG7P9E26PW|Me Talk Pretty On...|           5|
|A101DG7P9E26PW|        Cat's Cradle|           4|
|A101DG7P9E26PW|Middlesex [Unab

Percentage of review/score 

In [37]:
score = rates.select(["review/score"]).groupBy("review/score").count().sort("review/score")
score.show()

+------------+------+
|review/score| count|
+------------+------+
|           1| 13940|
|           2| 22281|
|           3| 60141|
|           4|140775|
|           5|280537|
+------------+------+



In [38]:
score.pandas_api().set_index("review/score").plot.pie(y="count")


c:\Users\ginny\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\pandas\__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


Book Data

In [39]:
books = spark.read.option('header', True)\
                  .csv('books_data.csv')

In [40]:
books.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+------------+
|               Title|         description|             authors|               image|         previewLink|           publisher| publishedDate|            infoLink|          categories|ratingsCount|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+------------+
|Its Only Art If I...|                null|    ['Julie Strain']|http://books.goog...|http://books.goog...|                null|          1996|http://books.goog...|['Comics & Graphi...|        null|
|Dr. Seuss: Americ...|"Philip Nel takes...| like that of Lew...| has changed lang...| giving us new wo...| inspiring artist...|['Philip Nel']|http://books.goog...|http://books.goog...|   A&C Black|
|Wonderful

In [41]:
books = books.drop("description", "image", "previewLink",\
                   "publisher", "publishedDate", "infoLink", "ratingsCount")

In [42]:
# drop rows with missing Title
books = books.na.drop(how="any", subset=["Title"])

In [43]:
# Define the regular expression pattern to match "str(list)"
pattern = r'^\[.*\]$'  # Assumes the list is enclosed in square brackets []

# Apply the transformation to categories and authors to get rid of outliers
books = books.withColumn('categories',
                         when(col('categories').rlike(
                             pattern), col('categories'))
                         .otherwise(None))
books = books.withColumn('authors',
                         when(col('authors').rlike(pattern), col('authors'))
                         .otherwise(None))

In [23]:
# if apply the cleaning first, the book title will not match title from rating
'''#change &#34; to "
books = books.withColumn('Title', trim(regexp_replace(col('Title'), '&#34;', '"')))
#replace "" with "
books = books.withColumn('Title', regexp_replace(col('Title'), '""', '"'))
books = books.withColumn('Title', regexp_replace(col('Title'), "''", "'"))
# Remove double quotes from the start and end of values
books = books.withColumn('Title', regexp_replace(col('Title'), '^"|"$', ''))
books = books.withColumn('Title', regexp_replace(col('Title'), "^'|'$", ""))
#remove beginning space
books = books.withColumn('Title', trim(regexp_replace(col('Title'), '^\\s+', '')))'''

'#change &#34; to "\nbooks = books.withColumn(\'Title\', trim(regexp_replace(col(\'Title\'), \'&#34;\', \'"\')))\n#replace "" with "\nbooks = books.withColumn(\'Title\', regexp_replace(col(\'Title\'), \'""\', \'"\'))\nbooks = books.withColumn(\'Title\', regexp_replace(col(\'Title\'), "\'\'", "\'"))\n# Remove double quotes from the start and end of values\nbooks = books.withColumn(\'Title\', regexp_replace(col(\'Title\'), \'^"|"$\', \'\'))\nbooks = books.withColumn(\'Title\', regexp_replace(col(\'Title\'), "^\'|\'$", ""))\n#remove beginning space\nbooks = books.withColumn(\'Title\', trim(regexp_replace(col(\'Title\'), \'^\\s+\', \'\')))'

In [44]:
# drop duplicate rows
books = books.distinct()
books.count()

212399

In [45]:
books.show(5)

+--------------------+--------------------+--------------------+
|               Title|             authors|          categories|
+--------------------+--------------------+--------------------+
|Whispers of the W...| ['Veronica Haddon']|         ['Fiction']|
|Sexual Arena and ...|['Edward J. Bardon']|        ['Feminism']|
|Mother S Table Fa...|['Jacqueline S. T...|  ['Social Science']|
|     White Rock Ways|                null|                null|
|Air Fare: Stories...|['Nickole Brown',...|['Literary Collec...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



Join 2 dataframe

In [47]:
# use fullouter join since evert data is important
df = rates.join(books, rates.Title == books.Title, "fullouter")\
           .select(books.Title, books.authors, books.categories,\
                   rates.User_id, rates["review/score"].alias("rates"))

In [48]:
# adjust the values in Title for better looking
# get rid of extra ""
df = df.withColumn('Title', regexp_replace(col('Title'), '""', '"'))
df = df.withColumn('Title', regexp_replace(col('Title'), '^"|"$', ''))

In [28]:
'''dd = rating.join(books, rating.Title == books.Title, "fullouter")\
              .select(books.Title.alias("booktitle"), books.authors, books.categories,\
               rating.Title,rating.User_id, rating["review/score"].alias("rates"))
#Every book in rating can be found in books
dd.filter(dd.Title.isNotNull() & dd.booktitle.isNull()).show()'''

'dd = rating.join(books, rating.Title == books.Title, "fullouter")              .select(books.Title.alias("booktitle"), books.authors, books.categories,               rating.Title,rating.User_id, rating["review/score"].alias("rates"))\n#Every book in rating can be found in books\ndd.filter(dd.Title.isNotNull() & dd.booktitle.isNull()).show()'

In [49]:
df.show(5,truncate=False)

+-----------------------------------------------------------------------+--------------------------------+----------+--------------+-----+
|Title                                                                  |authors                         |categories|User_id       |rates|
+-----------------------------------------------------------------------+--------------------------------+----------+--------------+-----+
|"Always ready!": The story of the United States Coast guard            |null                            |null      |null          |null |
|"Billboard" Book of Top 40 Albums                                      |null                            |null      |A2NJO6YE954DBH|4    |
|"Carefree" (R.K.O.Classic Screenplays)                                 |['Allan Scott', 'Ernest Pagano']|null      |A15LK8DSFQZZ52|5    |
|"Catch 'em alive Jack";: The life and adventures of an American pioneer|null                            |null      |null          |null |
|"Gentlemen prefer blondes;

In [50]:
df=df.distinct()

Output to csv file and split the validation data 

In [52]:
df.toPandas().set_index("User_id")\
  .to_csv("preprocess_20.csv", header=True, index="User_id")